In [1]:
from pymongo import MongoClient
import pandas as pd
from pprint import pprint
from flask import Flask, jsonify
import os

In [2]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [3]:
app = Flask(__name__)


In [4]:
# assign the uk_food database to a variable name
db = mongo['openChargeMap']


In [5]:
# review the collections in our database
print(db.list_collection_names())

['us_alt_fuel_stations', 'POI_List', 'EV_Make_Model', 'Alternative_Fueling_Stations']


In [6]:
# assign the collection to a variable
EV_Make_Model = db['EV_Make_Model']
POI_List = db['POI_List']
us_alt_fuel_stations = db['us_alt_fuel_stations']
Alternative_Fueling_Stations = db['Alternative_Fueling_Stations']

In [7]:
pprint(POI_List)
pprint(EV_Make_Model)
pprint(us_alt_fuel_stations)
pprint(Alternative_Fueling_Stations)

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'openChargeMap'), 'POI_List')
Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'openChargeMap'), 'EV_Make_Model')
Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'openChargeMap'), 'us_alt_fuel_stations')
Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'openChargeMap'), 'Alternative_Fueling_Stations')


In [8]:
def clean_address(address):
    # Example: Remove leading/trailing whitespace and capitalize letters
    return address.strip().title()


In [9]:
# Clean POI_List Collection
cleaned_poi_list = []
for doc in db['POI_List'].find():
    address_info = doc.get('AddressInfo', {})
    connections = doc.get('Connections', [])
    
    # Flatten AddressInfo object
    address_info_flat = {
        'AddressInfo_' + key: value
        for key, value in address_info.items()
    }
    
    # Extract required fields from Connections
    connection_data = []
    for connection in connections:
        connection_data.append({
            'ConnectionType_FormalName': connection.get('ConnectionType', {}).get('FormalName', ''),
            'ConnectionType_LevelID': connection.get('LevelID', '')
        })
    
    # Create a cleaned document
    cleaned_doc = {
        **address_info_flat,
        **{f'Connection_{index+1}_{key}': value for index, connection in enumerate(connection_data) for key, value in connection.items()},
        'NumberOfPoints': doc.get('NumberOfPoints', 0)
    }
    
    cleaned_poi_list.append(cleaned_doc)

# Create DataFrame from cleaned data
df_poi_list = pd.DataFrame(cleaned_poi_list)

# Display the DataFrame
print(df_poi_list.head())


   AddressInfo_ID                 AddressInfo_Title AddressInfo_AddressLine1  \
0          297854  Lewis Plaza - Tesla Supercharger           975 S Amity Rd   
1          297831                First Stop Branson   220 Branson Hills Pkwy   
2          297829    Fort Dodge Ford Lincoln Toyota          2723 5th Ave S.   
3          297828              Shell Council Bluffs         14151 Wabash Ave   
4          297825              Hy-Vee Grocery Store           2500 Humes Rd.   

  AddressInfo_AddressLine2 AddressInfo_Town AddressInfo_StateOrProvince  \
0                     None           Conway                          AR   
1                     None          Branson                          MO   
2                     None       Fort Dodge                        Iowa   
3                     None   Council Bluffs                        Iowa   
4                     None       Janesville                          WI   

  AddressInfo_Postcode  AddressInfo_CountryID  \
0                72

In [10]:
df_poi_list.head()

,AddressInfo_ID,AddressInfo_Title,AddressInfo_AddressLine1,AddressInfo_AddressLine2,AddressInfo_Town,AddressInfo_StateOrProvince,AddressInfo_Postcode,AddressInfo_CountryID,AddressInfo_Country,AddressInfo_Latitude,...,AddressInfo_DistanceUnit,Connection_1_ConnectionType_FormalName,Connection_1_ConnectionType_LevelID,NumberOfPoints,Connection_2_ConnectionType_FormalName,Connection_2_ConnectionType_LevelID,Connection_3_ConnectionType_FormalName,Connection_3_ConnectionType_LevelID,Connection_4_ConnectionType_FormalName,Connection_4_ConnectionType_LevelID
0,297854,Lewis Plaza - Tesla Supercharger,975 S Amity Rd,None,Conway,AR,72032,2,"{'ISOCode': 'US', 'ContinentCode': 'NA', 'ID':...",35.066785,...,0,SAE J3400,3.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN
1,297831,First Stop Branson,220 Branson Hills Pkwy,None,Branson,MO,65616,2,"{'ISOCode': 'US', 'ContinentCode': 'NA', 'ID':...",36.670296,...,0,IEC 62196-3 Configuration EE,3.0,1.0,IEC 62196-3 Configuration AA,3.0,SAE J1772-2009,2.0,NaN,NaN
2,297829,Fort Dodge Ford Lincoln Toyota,2723 5th Ave S.,None,Fort Dodge,Iowa,50501,2,"{'ISOCode': 'US', 'ContinentCode': 'NA', 'ID':...",42.500436,...,0,IEC 62196-3 Configuration EE,3.0,1.0,IEC 62196-3 Configuration EE,3.0,NaN,NaN,NaN,NaN
3,297828,Shell Council Bluffs,14151 Wabash Ave,None,Council Bluffs,Iowa,51503,2,"{'ISOCode': 'US', 'ContinentCode': 'NA', 'ID':...",41.220657,...,0,IEC 62196-3 Configuration EE,3.0,1.0,IEC 62196-3 Configuration AA,3.0,NaN,NaN,NaN,NaN
4,297825,Hy-Vee Grocery Store,2500 Humes Rd.,None,Janesville,WI,53545,2,"{'ISOCode': 'US', 'ContinentCode': 'NA', 'ID':...",42.718318,...,0,IEC 62196-3 Configuration EE,3.0,1.0,IEC 62196-3 Configuration AA,3.0,NaN,NaN,NaN,NaN


In [11]:
# Convert cleaned data to DataFrame
df_poi_list = pd.DataFrame(cleaned_poi_list)

# Display the DataFrame
print(df_poi_list.head())


   AddressInfo_ID                 AddressInfo_Title AddressInfo_AddressLine1  \
0          297854  Lewis Plaza - Tesla Supercharger           975 S Amity Rd   
1          297831                First Stop Branson   220 Branson Hills Pkwy   
2          297829    Fort Dodge Ford Lincoln Toyota          2723 5th Ave S.   
3          297828              Shell Council Bluffs         14151 Wabash Ave   
4          297825              Hy-Vee Grocery Store           2500 Humes Rd.   

  AddressInfo_AddressLine2 AddressInfo_Town AddressInfo_StateOrProvince  \
0                     None           Conway                          AR   
1                     None          Branson                          MO   
2                     None       Fort Dodge                        Iowa   
3                     None   Council Bluffs                        Iowa   
4                     None       Janesville                          WI   

  AddressInfo_Postcode  AddressInfo_CountryID  \
0                72

In [12]:
df_poi_list.head()

,AddressInfo_ID,AddressInfo_Title,AddressInfo_AddressLine1,AddressInfo_AddressLine2,AddressInfo_Town,AddressInfo_StateOrProvince,AddressInfo_Postcode,AddressInfo_CountryID,AddressInfo_Country,AddressInfo_Latitude,...,AddressInfo_DistanceUnit,Connection_1_ConnectionType_FormalName,Connection_1_ConnectionType_LevelID,NumberOfPoints,Connection_2_ConnectionType_FormalName,Connection_2_ConnectionType_LevelID,Connection_3_ConnectionType_FormalName,Connection_3_ConnectionType_LevelID,Connection_4_ConnectionType_FormalName,Connection_4_ConnectionType_LevelID
0,297854,Lewis Plaza - Tesla Supercharger,975 S Amity Rd,None,Conway,AR,72032,2,"{'ISOCode': 'US', 'ContinentCode': 'NA', 'ID':...",35.066785,...,0,SAE J3400,3.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN
1,297831,First Stop Branson,220 Branson Hills Pkwy,None,Branson,MO,65616,2,"{'ISOCode': 'US', 'ContinentCode': 'NA', 'ID':...",36.670296,...,0,IEC 62196-3 Configuration EE,3.0,1.0,IEC 62196-3 Configuration AA,3.0,SAE J1772-2009,2.0,NaN,NaN
2,297829,Fort Dodge Ford Lincoln Toyota,2723 5th Ave S.,None,Fort Dodge,Iowa,50501,2,"{'ISOCode': 'US', 'ContinentCode': 'NA', 'ID':...",42.500436,...,0,IEC 62196-3 Configuration EE,3.0,1.0,IEC 62196-3 Configuration EE,3.0,NaN,NaN,NaN,NaN
3,297828,Shell Council Bluffs,14151 Wabash Ave,None,Council Bluffs,Iowa,51503,2,"{'ISOCode': 'US', 'ContinentCode': 'NA', 'ID':...",41.220657,...,0,IEC 62196-3 Configuration EE,3.0,1.0,IEC 62196-3 Configuration AA,3.0,NaN,NaN,NaN,NaN
4,297825,Hy-Vee Grocery Store,2500 Humes Rd.,None,Janesville,WI,53545,2,"{'ISOCode': 'US', 'ContinentCode': 'NA', 'ID':...",42.718318,...,0,IEC 62196-3 Configuration EE,3.0,1.0,IEC 62196-3 Configuration AA,3.0,NaN,NaN,NaN,NaN


In [13]:
# Define new column names
new_column_names = {
    'AddressInfo_ID': 'Address_ID',
    'AddressInfo_Title': 'Title',
    'AddressInfo_AddressLine1': 'Address',
    'AddressInfo_AddressLine2': 'Address_Line2',
    'AddressInfo_Town': 'City',
    'AddressInfo_StateOrProvince': 'State',
    'AddressInfo_Postcode': 'ZIP Code',
    'Connection_1_ConnectionType_FormalName': 'Connection_1_Type',
    'Connection_1_ConnectionType_LevelID': 'Connection_1_LevelID',
    'NumberOfPoints': 'Number_Of_Points'
}

# Rename columns
df_poi_list.rename(columns=new_column_names, inplace=True)

# Display the DataFrame with renamed columns
print(df_poi_list.head())


   Address_ID                             Title                 Address  \
0      297854  Lewis Plaza - Tesla Supercharger          975 S Amity Rd   
1      297831                First Stop Branson  220 Branson Hills Pkwy   
2      297829    Fort Dodge Ford Lincoln Toyota         2723 5th Ave S.   
3      297828              Shell Council Bluffs        14151 Wabash Ave   
4      297825              Hy-Vee Grocery Store          2500 Humes Rd.   

  Address_Line2            City State ZIP Code  AddressInfo_CountryID  \
0          None          Conway    AR    72032                      2   
1          None         Branson    MO    65616                      2   
2          None      Fort Dodge  Iowa    50501                      2   
3          None  Council Bluffs  Iowa    51503                      2   
4          None      Janesville    WI    53545                      2   

                                 AddressInfo_Country  AddressInfo_Latitude  \
0  {'ISOCode': 'US', 'ContinentC

In [14]:
df_poi_list.head()

,Address_ID,Title,Address,Address_Line2,City,State,ZIP Code,AddressInfo_CountryID,AddressInfo_Country,AddressInfo_Latitude,...,AddressInfo_DistanceUnit,Connection_1_Type,Connection_1_LevelID,Number_Of_Points,Connection_2_ConnectionType_FormalName,Connection_2_ConnectionType_LevelID,Connection_3_ConnectionType_FormalName,Connection_3_ConnectionType_LevelID,Connection_4_ConnectionType_FormalName,Connection_4_ConnectionType_LevelID
0,297854,Lewis Plaza - Tesla Supercharger,975 S Amity Rd,None,Conway,AR,72032,2,"{'ISOCode': 'US', 'ContinentCode': 'NA', 'ID':...",35.066785,...,0,SAE J3400,3.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN
1,297831,First Stop Branson,220 Branson Hills Pkwy,None,Branson,MO,65616,2,"{'ISOCode': 'US', 'ContinentCode': 'NA', 'ID':...",36.670296,...,0,IEC 62196-3 Configuration EE,3.0,1.0,IEC 62196-3 Configuration AA,3.0,SAE J1772-2009,2.0,NaN,NaN
2,297829,Fort Dodge Ford Lincoln Toyota,2723 5th Ave S.,None,Fort Dodge,Iowa,50501,2,"{'ISOCode': 'US', 'ContinentCode': 'NA', 'ID':...",42.500436,...,0,IEC 62196-3 Configuration EE,3.0,1.0,IEC 62196-3 Configuration EE,3.0,NaN,NaN,NaN,NaN
3,297828,Shell Council Bluffs,14151 Wabash Ave,None,Council Bluffs,Iowa,51503,2,"{'ISOCode': 'US', 'ContinentCode': 'NA', 'ID':...",41.220657,...,0,IEC 62196-3 Configuration EE,3.0,1.0,IEC 62196-3 Configuration AA,3.0,NaN,NaN,NaN,NaN
4,297825,Hy-Vee Grocery Store,2500 Humes Rd.,None,Janesville,WI,53545,2,"{'ISOCode': 'US', 'ContinentCode': 'NA', 'ID':...",42.718318,...,0,IEC 62196-3 Configuration EE,3.0,1.0,IEC 62196-3 Configuration AA,3.0,NaN,NaN,NaN,NaN


In [15]:
# Check if 'AddressInfo_Country' column exists before dropping
if 'AddressInfo_Country' in df_poi_list.columns:
    # Drop the 'AddressInfo_Country' column
    df_poi_list.drop(columns=['AddressInfo_Country'], inplace=True)

# Display the DataFrame after dropping the column
print(df_poi_list.head())


   Address_ID                             Title                 Address  \
0      297854  Lewis Plaza - Tesla Supercharger          975 S Amity Rd   
1      297831                First Stop Branson  220 Branson Hills Pkwy   
2      297829    Fort Dodge Ford Lincoln Toyota         2723 5th Ave S.   
3      297828              Shell Council Bluffs        14151 Wabash Ave   
4      297825              Hy-Vee Grocery Store          2500 Humes Rd.   

  Address_Line2            City State ZIP Code  AddressInfo_CountryID  \
0          None          Conway    AR    72032                      2   
1          None         Branson    MO    65616                      2   
2          None      Fort Dodge  Iowa    50501                      2   
3          None  Council Bluffs  Iowa    51503                      2   
4          None      Janesville    WI    53545                      2   

   AddressInfo_Latitude  AddressInfo_Longitude  ... AddressInfo_DistanceUnit  \
0             35.066785       

In [16]:
# Columns to be deleted
columns_to_delete = [
    'AddressInfo_ContactTelephone2',
    'AddressInfo_ContactEmail',
    'AddressInfo_AccessComments',
    'AddressInfo_RelatedURL',
    'AddressInfo_Distance',
    'AddressInfo_DistanceUnit'
]

# Drop columns from df_poi_list DataFrame
df_poi_list.drop(columns=columns_to_delete, inplace=True)
# Rename columns in df_poi_list DataFrame
df_poi_list.rename(columns={'AddressInfo_Latitude': 'Latitude', 'AddressInfo_Longitude': 'Longitude'}, inplace=True)

# Display the DataFrame after renaming columns
print(df_poi_list.head())

# Display the DataFrame after dropping columns
print(df_poi_list.head())


   Address_ID                             Title                 Address  \
0      297854  Lewis Plaza - Tesla Supercharger          975 S Amity Rd   
1      297831                First Stop Branson  220 Branson Hills Pkwy   
2      297829    Fort Dodge Ford Lincoln Toyota         2723 5th Ave S.   
3      297828              Shell Council Bluffs        14151 Wabash Ave   
4      297825              Hy-Vee Grocery Store          2500 Humes Rd.   

  Address_Line2            City State ZIP Code  AddressInfo_CountryID  \
0          None          Conway    AR    72032                      2   
1          None         Branson    MO    65616                      2   
2          None      Fort Dodge  Iowa    50501                      2   
3          None  Council Bluffs  Iowa    51503                      2   
4          None      Janesville    WI    53545                      2   

    Latitude  Longitude AddressInfo_ContactTelephone1  \
0  35.066785 -92.413765                          None

In [17]:
# Clean Alternative_Fueling_Stations Collection
cleaned_alt_fueling_stations = []
for doc in db['Alternative_Fueling_Stations'].find():
    cleaned_doc = {
        'access_code': doc.get('access_code', ''),
        'station_name': doc.get('station_name', ''),
        'geocode_status': doc.get('geocode_status', ''),
        'city': doc.get('city', ''),
        'state': doc.get('state', ''),
        'street_address': doc.get('street_address', ''),
        'zip': doc.get('zip', ''),
        'groups_with_access_code': doc.get('groups_with_access_code', ''),

        'longitude': doc.get('longitude', ''),
        'latitude': doc.get('latitude', '')
    }
    cleaned_alt_fueling_stations.append(cleaned_doc)

# Create DataFrame from cleaned data
df_alt_fueling_stations = pd.DataFrame(cleaned_alt_fueling_stations)

# Display the DataFrame
print(df_alt_fueling_stations.head())


  access_code                                  station_name geocode_status  \
0     private          Spire - Montgomery Operations Center          200-9   
1     private  Metropolitan Atlanta Rapid Transit Authority          200-8   
2     private                         United Parcel Service          200-9   
3      public                    Arkansas Oklahoma Gas Corp          200-9   
4      public    Clean Energy - Logan International Airport            GPS   

          city state        street_address    zip  \
0   Montgomery    AL      2951 Chestnut St  36107   
1      Atlanta    GA   2424 Piedmont Rd NE  30324   
2      Atlanta    GA  270 Marvin Miller Dr  30336   
3   Fort Smith    AR     2100 S Waldron Rd  72903   
4  East Boston    MA   1000 Cottage St Ext  02128   

             groups_with_access_code          longitude   latitude  
0                            Private         -86.267021  32.367916  
1          Private - Government only         -84.367461  33.821911  
2    

In [18]:
# Define more descriptive column names
descriptive_column_names = {
    'access_code': 'Access Code',
    'station_name': 'Title',
    'geocode_status': 'Geocode Status',
    'city': 'City',
    'state': 'State',
    'street_address': 'Address',
    'zip': 'ZIP Code',
    'groups_with_access_code': 'Groups With Access Code',
    'longitude': 'Longitude',
    'latitude': 'Latitude'
}

# Rename columns in df_alt_fueling_stations DataFrame
df_alt_fueling_stations.rename(columns=descriptive_column_names, inplace=True)

# Display the DataFrame with more descriptive column names
print(df_alt_fueling_stations.head())



  Access Code                                         Title Geocode Status  \
0     private          Spire - Montgomery Operations Center          200-9   
1     private  Metropolitan Atlanta Rapid Transit Authority          200-8   
2     private                         United Parcel Service          200-9   
3      public                    Arkansas Oklahoma Gas Corp          200-9   
4      public    Clean Energy - Logan International Airport            GPS   

          City State               Address ZIP Code  \
0   Montgomery    AL      2951 Chestnut St    36107   
1      Atlanta    GA   2424 Piedmont Rd NE    30324   
2      Atlanta    GA  270 Marvin Miller Dr    30336   
3   Fort Smith    AR     2100 S Waldron Rd    72903   
4  East Boston    MA   1000 Cottage St Ext    02128   

             Groups With Access Code          Longitude   Latitude  
0                            Private         -86.267021  32.367916  
1          Private - Government only         -84.367461  33.82

In [19]:
df_alt_fueling_stations.head()

,Access Code,Title,Geocode Status,City,State,Address,ZIP Code,Groups With Access Code,Longitude,Latitude
0,private,Spire - Montgomery Operations Center,200-9,Montgomery,AL,2951 Chestnut St,36107,Private,-86.267021,32.367916
1,private,Metropolitan Atlanta Rapid Transit Authority,200-8,Atlanta,GA,2424 Piedmont Rd NE,30324,Private - Government only,-84.367461,33.821911
2,private,United Parcel Service,200-9,Atlanta,GA,270 Marvin Miller Dr,30336,Private,-84.543822,33.760256
3,public,Arkansas Oklahoma Gas Corp,200-9,Fort Smith,AR,2100 S Waldron Rd,72903,Public - Credit card at all times,-94.375338,35.362213
4,public,Clean Energy - Logan International Airport,GPS,East Boston,MA,1000 Cottage St Ext,02128,Public - Credit card at all times,-71.0265486057434,42.374706


In [20]:
# Clean us_alt_fuel_stations Collection
cleaned_us_alt_fuel_stations = []
for doc in db['us_alt_fuel_stations'].find():
    cleaned_doc = {
        'Fuel Type Code': doc.get('Fuel Type Code', ''),
        'Station Name': doc.get('Station Name', ''),
        'Street Address': doc.get('Street Address', ''),
        'City': doc.get('City', ''),
        'State': doc.get('State', ''),
        'ZIP': doc.get('ZIP', ''),
        'Groups With Access Code': doc.get('Groups With Access Code', ''),
        'Country': doc.get('Country', ''),
        'Station Phone': doc.get('Station Phone', ''),
        'Status Code': doc.get('Status Code', ''),
        'GroupsWith Access Code': doc.get('GroupsWith Access Code', ''),
        'EV Level2 EVSE Num': doc.get('EV Level2 EVSE Num', ''),
        'EV Network': doc.get('EV Network', ''),
        'GeoCode Status': doc.get('Geocode Status', ''),
        'Latitude': doc.get('Latitude', ''),
        'Longitude': doc.get('Longitude', ''),
        'Access Code': doc.get('Access Code', ''),
        'Facility Type': doc.get('Facility Type', ''),
        'EV Connector Types': doc.get('EV Connector Types', '')
    }
    cleaned_us_alt_fuel_stations.append(cleaned_doc)

# Create DataFrame from cleaned data
df_us_alt_fuel_stations = pd.DataFrame(cleaned_us_alt_fuel_stations)

# Display the DataFrame
print(df_us_alt_fuel_stations.head())


  Fuel Type Code                    Station Name               Street Address  \
0           ELEC   Los Angeles Convention Center           1201 S Figueroa St   
1           ELEC  California Air Resources Board             9530 Telstar Ave   
2           ELEC          Scripps Green Hospital      10666 N Torrey Pines Rd   
3           ELEC      San Diego Wild Animal Park  15500 San Pasqual Valley Rd   
4           ELEC                   Galpin Motors            15421 Roscoe Blvd   

          City State    ZIP Groups With Access Code Country Station Phone  \
0  Los Angeles    CA  90015                  Public      US  213-741-1151   
1     El Monte    CA  91731                  Public      US  626-575-6800   
2     La Jolla    CA  92037                  Public      US  858-554-9100   
3    Escondido    CA  92027                  Public      US  760-747-8702   
4    Sepulveda    CA  91343     Public - Call ahead      US  800-256-6219   

  Status Code GroupsWith Access Code EV Level2 EVS

In [21]:
df_us_alt_fuel_stations = df_us_alt_fuel_stations[df_us_alt_fuel_stations['Country'] != 'CA']

In [22]:
# Define column name mapping dictionary
column_mapping = {
    'Fuel Type Code': 'Fuel Type',
    'Station Name': 'Title',
    'Street Address': 'Address',
    'City': 'City',
    'State': 'State',
    'ZIP': 'ZIP Code',
    'Groups With Access Code': 'Groups With Access Code',
    'Station Phone': 'Phone Number',  # Assuming Station Phone is the contact phone
    'Status Code': 'StatusType_Title',
    'GroupsWith Access Code': '',
    'EV Level2 EVSE Num': 'Charging Levels',
    'EV Network': 'OperatorTitle',
    'GeoCode Status': 'Geocode Status',
    'Latitude': 'Latitude',
    'Longitude': 'Longitude',
    'Access Code': 'Access Code',
    'Facility Type': 'Facility Type',
    'EV Connector Types': 'Connector Type'
}

# Rename columns in df_us_alt_fuel_stations DataFrame
df_us_alt_fuel_stations.rename(columns=column_mapping, inplace=True)

# Display the DataFrame with renamed columns
print(df_us_alt_fuel_stations.head())


  Fuel Type                           Title                      Address  \
0      ELEC   Los Angeles Convention Center           1201 S Figueroa St   
1      ELEC  California Air Resources Board             9530 Telstar Ave   
2      ELEC          Scripps Green Hospital      10666 N Torrey Pines Rd   
3      ELEC      San Diego Wild Animal Park  15500 San Pasqual Valley Rd   
4      ELEC                   Galpin Motors            15421 Roscoe Blvd   

          City State ZIP Code Groups With Access Code Country  Phone Number  \
0  Los Angeles    CA    90015                  Public      US  213-741-1151   
1     El Monte    CA    91731                  Public      US  626-575-6800   
2     La Jolla    CA    92037                  Public      US  858-554-9100   
3    Escondido    CA    92027                  Public      US  760-747-8702   
4    Sepulveda    CA    91343     Public - Call ahead      US  800-256-6219   

  StatusType_Title UsageType_Title NumberOfPoints  OperatorTitle  \


In [23]:
# Print all column names and their data types in df_poi_list
for column, dtype in df_poi_list.dtypes.items():
    print(f"Column: {column}, Data Type: {dtype}")


Column: Address_ID, Data Type: int64
Column: Title, Data Type: object
Column: Address, Data Type: object
Column: Address_Line2, Data Type: object
Column: City, Data Type: object
Column: State, Data Type: object
Column: ZIP Code, Data Type: object
Column: AddressInfo_CountryID, Data Type: int64
Column: Latitude, Data Type: float64
Column: Longitude, Data Type: float64
Column: AddressInfo_ContactTelephone1, Data Type: object
Column: Connection_1_Type, Data Type: object
Column: Connection_1_LevelID, Data Type: float64
Column: Number_Of_Points, Data Type: float64
Column: Connection_2_ConnectionType_FormalName, Data Type: object
Column: Connection_2_ConnectionType_LevelID, Data Type: float64
Column: Connection_3_ConnectionType_FormalName, Data Type: object
Column: Connection_3_ConnectionType_LevelID, Data Type: float64
Column: Connection_4_ConnectionType_FormalName, Data Type: object
Column: Connection_4_ConnectionType_LevelID, Data Type: float64


In [24]:
# Convert 'Longitude' and 'Latitude' columns to numeric data types in all DataFrames
df_poi_list['Longitude'] = pd.to_numeric(df_poi_list['Longitude'], errors='coerce')
df_poi_list['Latitude'] = pd.to_numeric(df_poi_list['Latitude'], errors='coerce')

df_alt_fueling_stations['Longitude'] = pd.to_numeric(df_alt_fueling_stations['Longitude'], errors='coerce')
df_alt_fueling_stations['Latitude'] = pd.to_numeric(df_alt_fueling_stations['Latitude'], errors='coerce')

df_us_alt_fuel_stations['Longitude'] = pd.to_numeric(df_us_alt_fuel_stations['Longitude'], errors='coerce')
df_us_alt_fuel_stations['Latitude'] = pd.to_numeric(df_us_alt_fuel_stations['Latitude'], errors='coerce')

# Display data types of 'Longitude' and 'Latitude' columns in all DataFrames
print("df_poi_list:")
print(df_poi_list[['Longitude', 'Latitude']].dtypes)
print("\ndf_alt_fueling_stations:")
print(df_alt_fueling_stations[['Longitude', 'Latitude']].dtypes)
print("\ndf_us_alt_fuel_stations:")
print(df_us_alt_fuel_stations[['Longitude', 'Latitude']].dtypes)


df_poi_list:
Longitude    float64
Latitude     float64
dtype: object

df_alt_fueling_stations:
Longitude    float64
Latitude     float64
dtype: object

df_us_alt_fuel_stations:
Longitude    float64
Latitude     float64
dtype: object


In [25]:
# Merge DataFrames based on longitude and latitude using outer join
merged_df = pd.merge(df_poi_list, df_alt_fueling_stations, on=['Longitude', 'Latitude'], how='outer')

# Display the merged DataFrame
print(merged_df.head())


   Address_ID                           Title_x               Address_x  \
0    297854.0  Lewis Plaza - Tesla Supercharger          975 S Amity Rd   
1    297831.0                First Stop Branson  220 Branson Hills Pkwy   
2    297829.0    Fort Dodge Ford Lincoln Toyota         2723 5th Ave S.   
3    297828.0              Shell Council Bluffs        14151 Wabash Ave   
4    297825.0              Hy-Vee Grocery Store          2500 Humes Rd.   

  Address_Line2          City_x State_x ZIP Code_x  AddressInfo_CountryID  \
0          None          Conway      AR      72032                    2.0   
1          None         Branson      MO      65616                    2.0   
2          None      Fort Dodge    Iowa      50501                    2.0   
3          None  Council Bluffs    Iowa      51503                    2.0   
4          None      Janesville      WI      53545                    2.0   

    Latitude  Longitude  ... Connection_4_ConnectionType_FormalName  \
0  35.066785 -9

In [26]:
merged_df.head()

,Address_ID,Title_x,Address_x,Address_Line2,City_x,State_x,ZIP Code_x,AddressInfo_CountryID,Latitude,Longitude,...,Connection_4_ConnectionType_FormalName,Connection_4_ConnectionType_LevelID,Access Code,Title_y,Geocode Status,City_y,State_y,Address_y,ZIP Code_y,Groups With Access Code
0,297854.0,Lewis Plaza - Tesla Supercharger,975 S Amity Rd,None,Conway,AR,72032,2.0,35.066785,-92.413765,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,297831.0,First Stop Branson,220 Branson Hills Pkwy,None,Branson,MO,65616,2.0,36.670296,-93.225603,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,297829.0,Fort Dodge Ford Lincoln Toyota,2723 5th Ave S.,None,Fort Dodge,Iowa,50501,2.0,42.500436,-94.159283,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,297828.0,Shell Council Bluffs,14151 Wabash Ave,None,Council Bluffs,Iowa,51503,2.0,41.220657,-95.817930,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,297825.0,Hy-Vee Grocery Store,2500 Humes Rd.,None,Janesville,WI,53545,2.0,42.718318,-88.992866,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
print(merged_df.head())

   Address_ID                           Title_x               Address_x  \
0    297854.0  Lewis Plaza - Tesla Supercharger          975 S Amity Rd   
1    297831.0                First Stop Branson  220 Branson Hills Pkwy   
2    297829.0    Fort Dodge Ford Lincoln Toyota         2723 5th Ave S.   
3    297828.0              Shell Council Bluffs        14151 Wabash Ave   
4    297825.0              Hy-Vee Grocery Store          2500 Humes Rd.   

  Address_Line2          City_x State_x ZIP Code_x  AddressInfo_CountryID  \
0          None          Conway      AR      72032                    2.0   
1          None         Branson      MO      65616                    2.0   
2          None      Fort Dodge    Iowa      50501                    2.0   
3          None  Council Bluffs    Iowa      51503                    2.0   
4          None      Janesville      WI      53545                    2.0   

    Latitude  Longitude  ... Connection_4_ConnectionType_FormalName  \
0  35.066785 -9

In [28]:
# Merge DataFrames based on columns with the same column names
final_merged_df = pd.merge(df_alt_fueling_stations, df_us_alt_fuel_stations, how='outer')

# Display the merged DataFrame
print(final_merged_df.head())


  Access Code                                         Title Geocode Status  \
0     private          Spire - Montgomery Operations Center          200-9   
1     private  Metropolitan Atlanta Rapid Transit Authority          200-8   
2     private                         United Parcel Service          200-9   
3      public                    Arkansas Oklahoma Gas Corp          200-9   
4      public    Clean Energy - Logan International Airport            GPS   

          City State               Address ZIP Code  \
0   Montgomery    AL      2951 Chestnut St    36107   
1      Atlanta    GA   2424 Piedmont Rd NE    30324   
2      Atlanta    GA  270 Marvin Miller Dr    30336   
3   Fort Smith    AR     2100 S Waldron Rd    72903   
4  East Boston    MA   1000 Cottage St Ext    02128   

             Groups With Access Code  Longitude   Latitude  ... Country  \
0                            Private -86.267021  32.367916  ...     NaN   
1          Private - Government only -84.367461  3

In [29]:
final_merged_df.head()

,Access Code,Title,Geocode Status,City,State,Address,ZIP Code,Groups With Access Code,Longitude,Latitude,...,Country,Phone Number,StatusType_Title,UsageType_Title,NumberOfPoints,OperatorTitle,Geocode_Status,Access_Code,Facility_Type,Connector_Type
0,private,Spire - Montgomery Operations Center,200-9,Montgomery,AL,2951 Chestnut St,36107,Private,-86.267021,32.367916,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,private,Metropolitan Atlanta Rapid Transit Authority,200-8,Atlanta,GA,2424 Piedmont Rd NE,30324,Private - Government only,-84.367461,33.821911,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,private,United Parcel Service,200-9,Atlanta,GA,270 Marvin Miller Dr,30336,Private,-84.543822,33.760256,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,public,Arkansas Oklahoma Gas Corp,200-9,Fort Smith,AR,2100 S Waldron Rd,72903,Public - Credit card at all times,-94.375338,35.362213,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,public,Clean Energy - Logan International Airport,GPS,East Boston,MA,1000 Cottage St Ext,02128,Public - Credit card at all times,-71.026549,42.374706,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [30]:
# Print all column names and their data types in the merged DataFrame
for column, dtype in final_merged_df.dtypes.items():
    print(f"Column: {column}, Data Type: {dtype}")


Column: Access Code, Data Type: object
Column: Title, Data Type: object
Column: Geocode Status, Data Type: object
Column: City, Data Type: object
Column: State, Data Type: object
Column: Address, Data Type: object
Column: ZIP Code, Data Type: object
Column: Groups With Access Code, Data Type: object
Column: Longitude, Data Type: float64
Column: Latitude, Data Type: float64
Column: Fuel Type, Data Type: object
Column: Country, Data Type: object
Column: Phone Number, Data Type: object
Column: StatusType_Title, Data Type: object
Column: UsageType_Title, Data Type: object
Column: NumberOfPoints, Data Type: object
Column: OperatorTitle, Data Type: object
Column: Geocode_Status, Data Type: object
Column: Access_Code, Data Type: object
Column: Facility_Type, Data Type: object
Column: Connector_Type, Data Type: object


In [31]:
# Remove non-numeric characters from 'ZIP Code' column
final_merged_df['ZIP Code'] = final_merged_df['ZIP Code'].str.replace(r'\D+', '', regex=True)

# Convert 'ZIP Code' column to integer data type
final_merged_df['ZIP Code'] = pd.to_numeric(final_merged_df['ZIP Code'], errors='coerce', downcast='integer')

# Print all column names and their data types in the final_merged_df DataFrame
for column, dtype in final_merged_df.dtypes.items():
    print(f"Column: {column}, Data Type: {dtype}")


Column: Access Code, Data Type: object
Column: Title, Data Type: object
Column: Geocode Status, Data Type: object
Column: City, Data Type: object
Column: State, Data Type: object
Column: Address, Data Type: object
Column: ZIP Code, Data Type: float64
Column: Groups With Access Code, Data Type: object
Column: Longitude, Data Type: float64
Column: Latitude, Data Type: float64
Column: Fuel Type, Data Type: object
Column: Country, Data Type: object
Column: Phone Number, Data Type: object
Column: StatusType_Title, Data Type: object
Column: UsageType_Title, Data Type: object
Column: NumberOfPoints, Data Type: object
Column: OperatorTitle, Data Type: object
Column: Geocode_Status, Data Type: object
Column: Access_Code, Data Type: object
Column: Facility_Type, Data Type: object
Column: Connector_Type, Data Type: object


In [32]:

# Get the current directory of the notebook
current_dir = os.getcwd()

# Define the file path relative to the resource folder
file_path = os.path.join(current_dir, 'resource', 'final_merged_data.json')

In [33]:
# Convert DataFrame to JSON format
json_data = final_merged_df.to_json(orient='records', indent=4)

# Write JSON data to a file
with open(file_path, 'w') as file:
    file.write(json_data)

print(f"JSON data has been exported to {file_path}")


JSON data has been exported to /Users/albertoalvarado/Documents/utsa/UTSA.Lessons/Project3/resource/final_merged_data.json


In [35]:
df_alt_fueling_stations.count()

Access Code                82869
Title                      82869
Geocode Status             82869
City                       82869
State                      82869
Address                    82869
ZIP Code                   82869
Groups With Access Code    82869
Longitude                  82869
Latitude                   82869
dtype: int64

In [36]:
df_us_alt_fuel_stations.count()

Fuel Type                  42878
Title                      42878
Address                    42878
City                       42878
State                      42878
ZIP Code                   42878
Groups With Access Code    42878
Country                    42878
Phone Number               42878
StatusType_Title           42878
UsageType_Title            42878
NumberOfPoints             42878
OperatorTitle              42878
Geocode_Status             42878
Latitude                   42878
Longitude                  42878
Access_Code                42878
Facility_Type              42878
Connector_Type             42878
dtype: int64

In [37]:
final_merged_df.count()

Access Code                 82881
Title                      106069
Geocode Status              82881
City                       106069
State                      106069
Address                    106069
ZIP Code                   106024
Groups With Access Code    106069
Longitude                  106069
Latitude                   106069
Fuel Type                   42900
Country                     42900
Phone Number                42900
StatusType_Title            42900
UsageType_Title             42900
NumberOfPoints              42900
OperatorTitle               42900
Geocode_Status              42900
Access_Code                 42900
Facility_Type               42900
Connector_Type              42900
dtype: int64